In [1]:
from packages import *

# Torsionsweicher Träggerrost

Das folgende Beispiel ist aus @marti_baustatik_2014 entnommen. Dieses dient als einführendes Beispiel in die Modellierung von Trägerrosten mit dem Federmodell. In der @fig-marti_rost_gr ist der Grundriss des Trägerrosts dargestellt. Es handelt sich um insgesamt 16 torsionsweiche Stäbe, welche an den Enden eingespannt sind.

![Grundriss des torsionsweichen Trägerrosts](../imgs/marti_traegerrost_gr.png){#fig-marti_rost_gr}

Im Beispiel wird eine analytische Lösung zur Traglast aufgezeigt. Das Ziel ist es, mittels dem Federmodell eine numerische Lösung des unteren Grenzwerts der Traglast zu erhalten.






## Analytische Lösung

Die Analytische Lösung basiert auf dem Traglastverfahren. Mittels einem zulässigen Mechanismus wird ein oberer Grenzwert der Traglast hergeleitet. In der @fig-marti_mechanismus sind zwei Stäbe dargestellt. Aus symmetriegründen lässt sich der obere Grenzwert des gesamten Trägerrosts anhand dieser Darstellung ermitteln.

![Mechanismus eines Stabs des Trägerrosts](../imgs/marti_traegerrost_mechanismus.png){#fig-marti_mechanismus}



Die äussere Arbeit $W_a$ des dargestellten Systems in @fig-marti_mechanismus beträgt dabei für die am Rand gelegenen Stäbe (Punkte 2'45):

$$
W_{a,2'45} = 4 \cdot \left( Q \cdot \frac{1}{3} + Q \cdot \frac{1}{9} \right)
$$

Und für die Innenträger:

$$
W_{a,123} = 4 \cdot \left( Q \cdot 1 + Q \cdot \frac{1}{3} \right)
$$


Sowie beträgt die innere Arbeit:

$$
W_i = 8 \cdot \left(M_u' + M_u\right) \cdot \left(\frac{1}{3l} +\frac{1}{9l}\right)
$$

Durch das abschliessende Gleichsetzen der Arbeiten und das Lösen nach $Q$ folgt der obere Grenzwert der Traglast zu:

$$
Q = \frac{M_u + M_u'}{2l}
$$

Die Plastizitätskontrolle ist in der @fig-marti_plastizitaetskontrolle gezeigt. Dabei wird eine Lastverteilung von je einer Hälfte $Q$ in $x$ und $y$ Richtung angenommen.

![Plastizitätskontrolle anhand der Schnittgrössen](../imgs/marti_traegerrost_schnittgroessen.png){#fig-marti_plastizitaetskontrolle}

Aus der Plastizitätskontrolle geht heraus, dass der Biegewiderstand nirgends überschritten wird, somit deckt sich der untere und obere Grenzwert der Traglast, sprich die Traglast $Q_u$ wurde gefunden.


$$
Q_u = \frac{M_u + M_u'}{2l}
$$



## Numerische Lösung

Abschliessend wird die analytische Lösung mit der numerischen Lösung verglichen. Dabei wird ein Federmodell erstellt.
Zunächst sind die Variablen mit numerischen Werten zu substituieren:

In [2]:
l = 1*m 
M_u = 100*kN*m
M_apos_u = 100*kN*m

Q_u = 0.5*(M_u + M_apos_u) /l

put_out()

$$
\begin{aligned}
l & = 1 \ \mathrm{m} \quad & M_{u} & = 100 \ \mathrm{kN} \cdot \mathrm{m} \quad & M'_{u} & = 100 \ \mathrm{kN} \cdot \mathrm{m} \\ 
Q_{u} & = 100.0 \ \mathrm{kN} \quad &  \quad &  
 \end{aligned}
$$

Der Trägerrost wurde mittels dem Federmodell nachmodelliert. Dabei sind biegestarre, jedoch torsionsweiche Stäbe mittels Stabendgelenken gekoppelt. Dargestellt in der @fig-AxisVM_marti_geom.

![Statisches Modell in AxisVM des Trägerrosts](../imgs/marti_fem_geom.png){#fig-fem_marti_geom}

![Gelenkdefinition in AxisVM des Trägerrosts](../imgs/marti_fem_gelenk.png){#fig-AxisVM_marti_gelenk}

Den Gelenken wurde die Defintion gemäss @fig-AxisVM_marti_gelenk hinterlegt. Dabei wurde der Biegewiderstand in positiver und negativer Dimension angesetzt, ab dem Erreichen des Biegewiderstands fliesst das Gelenk. Dies führt dazu, dass sich die Biegemomente umlagern, sprich der Biegewiderstand auch in Trägermitte erreicht werden kann.

![Biegemomente in AxisVM des Trägerrosts](../imgs/marti_fem_my.png){#fig-AxisVM_marti_res}

Mit der Belastung aus dem unteren Grenzwert stellen sich exakt die Extremwerte der Biegemomente $M_u$ und $M'_u$ in den innen liegenden Stäben ein. Das Federmodell liefert den exakten unteren Grenzwert der Traglast. Dargestell in der @fig-AxisVM_marti_res.

# Torsionsteifer Trägerrost

In [3]:
Q_lower_comma_tors = 1.206 * Q_u

put_out()

$$
\begin{aligned}
Q_{lower , tors} & = 120.6 \ \mathrm{kN} \end{aligned}
$$

In [4]:
0.9*110

99.0